In [16]:
# Import necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import (mean_absolute_error, mean_squared_error, r2_score, accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report, roc_auc_score, roc_curve)
import xgboost as xgb

import joblib
import warnings
warnings.filterwarnings('ignore')
import os

In [17]:
# Get the datasets and initialize dictionaries

BASE_DIR = os.path.dirname(os.path.dirname(os.getcwd()))# Current working directory of the notebook
data_dir = os.path.join(BASE_DIR, 'data', 'engineered_data')

print("Using data directory:", data_dir)

models = {}
scalers = {}
results = {}



Using data directory: /home/darlenewendie/PycharmProjects/Intelligent-Manganese-Processing-Plant-Optimization/data/engineered_data


In [18]:
# Loading engineered datasets
print("LOADING ENGINEERED DATASETS")
datasets = {}

files = {
    'separation': 'engineered_separation.csv',
    'floatation': 'engineered_flotation.csv',
    'equipment':'engineered_equipment.csv',
    'crushing': 'engineered_crushing.csv',
    'dms': 'engineered_dms.csv'
}

for name, filename in files.items():
    filepath  = os.path.join(data_dir, filename)
    try:
        df= pd.read_csv(filepath, parse_dates=['timestamp'])
        datasets[name] = df
        print(f"Loaded {name}:  {len(df): }records and {len(df.columns) :} columns")
    except FileNotFoundError:
        print(f"File Not found:  {filepath}")
    except Exception as e:
        print(f"Error Loading: {name}: {str(e)}")

print(f"Total datasets loaded: {len(datasets)}")


LOADING ENGINEERED DATASETS
Loaded separation:   12000records and 32 columns
Loaded floatation:   12000records and 46 columns
Loaded equipment:   8000records and 27 columns
Loaded crushing:   15000records and 26 columns
Loaded dms:   8000records and 29 columns
Total datasets loaded: 5


In [15]:
# MODEL 1: RECOVERY PREDICTION

def prepare_recovery_data(datasets):
    print("MODEL 1: RECOVERY - PREDICTION - DATA PREPARATION")
    print("="*80)

    # Use multiple datasets for recovery prediction
    recovery_data = []

    # Gravity separation in recovery
    if 'separation' in datasets:
        sep_df = datasets['separation'].copy()

        # Select the Features for gravity separation to e used in ore-recovery prediction
        gravity_features = [
            # Ore characteristics
            'feed_grade_pct' , 'ore_quality_index', 'mn_to_fe_ratio',
            # Process parameters
            'spiral_speed_rpm', 'wash_water_m3h', 'feed_density_pct_solids',
            'spiral_speed_deviation', 'water_to_solids',
            # Equipment/ performance
            'spiral_concentrate_grade_pct', 'upgrade_ratio', 'separation_sharpness',
            #Magnetic separation
            'magnetic_intensity_t', ' belt_speed_ms', 'magnetic_intensity_effect',
            # Engineered datasets
            'grade_recovery_product', 'enrichment_index', 'separation_selectivity',
            'overall_enrichment', 'combined_efficiency'
        ]

        # Check which features exist
        available_gravity_features = [f for f in gravity_features if f in sep_df.columns]

        if 'overall_recovery' in sep_df.columns and len(available_gravity_features) > 5:
            gravity_df = sep_df[available_gravity_features + ['overall_recovery']].copy()
            gravity_df['circuit_type'] = 'gravity'
            recovery_data.append(gravity_df)
            print(f"✓ Gravity separation: {len(gravity_df)} records, {len(available_gravity_features)} features")

    # Flotation recovery
    if 'flotation' in datasets:
        flot_df = datasets['flotation'].copy()

        flotation_features = [
            # Feed characteristics
            'feed_grade_pct', 'ore_quality_index',
            # Reagent parameters
            'collector_dosage_gt', 'frother_dosage_gt', 'reagent_efficiency',
            'collector_intensity', 'collector_to_frother_ratio',
            # pH control
            'ph_value', 'ph_deviation_from_optimal', 'ph_in_optimal_range',
            # Process parameters
            'pulp_density_pct_solids', 'air_flow_m3_min', 'residence_time_min',
            'air_to_solids_ratio', 'flotation_kinetics_factor',
            # Equipment health (if available)
            'cell_health_score', 'blower_health_score',
            'cell_health_recovery_product', 'blower_efficiency_factor',
            # Performance indicators
            'concentrate_grade_pct', 'froth_stability_index', 'froth_loading'
        ]

        available_flotation_features = [f for f in flotation_features if f in flot_df.columns]

        if 'flotation_recovery' in flot_df.columns and len(available_flotation_features) > 5:
            flotation_df = flot_df[available_flotation_features + ['flotation_recovery']].copy()
            flotation_df.rename(columns={'flotation_recovery': 'overall_recovery'}, inplace=True)
            flotation_df['circuit_type'] = 'flotation'
            recovery_data.append(flotation_df)
            print(f"✓ Flotation: {len(flotation_df)} records, {len(available_flotation_features)} features")

    # DMS recovery
    if 'dms' in datasets:
        dms_df = datasets['dms'].copy()

        dms_features = [
            'feed_grade_pct', 'feed_size_mm', 'media_density_sg', 'cyclone_pressure_kpa',
            'density_differential', 'separation_sharpness_dms', 'media_efficiency',
            'pressure_utilization', 'media_consumption_efficiency',
            'size_suitability_for_dms', 'coarse_fraction_ratio',
            'cyclone_health_score', 'cyclone_health_efficiency_product',
            'sink_grade_pct', 'dms_upgrade_factor'
        ]

        available_dms_features = [f for f in dms_features if f in dms_df.columns]

        if 'dms_recovery' in dms_df.columns and len(available_dms_features) > 5:
            dms_data = dms_df[available_dms_features + ['dms_recovery']].copy()
            dms_data.rename(columns={'dms_recovery': 'overall_recovery'}, inplace=True)
            dms_data['circuit_type'] = 'dms'
            recovery_data.append(dms_data)
            print(f"✓ DMS: {len(dms_data)} records, {len(available_dms_features)} features")

    if not recovery_data:
        raise ValueError("No recovery data available!")

    # Combine all recovery data
    combined_recovery = pd.concat(recovery_data, ignore_index=True)

    print(f"\n✓ Combined dataset: {combined_recovery.shape}")
    print(f"  Target variable: overall_recovery")
    print(f"  Range: {combined_recovery['overall_recovery'].min():.3f} - {combined_recovery['overall_recovery'].max():.3f}")
    print(f"  Mean: {combined_recovery['overall_recovery'].mean():.3f}")

    return combined_recovery

def train_recovery_model(self, recovery_data):
    """Train recovery prediction model"""
    print("\n" + "="*80)
    print("MODEL 1: TRAINING RECOVERY PREDICTION MODEL")
    print("="*80)

    # Handle missing values
    print("\n1. Handling Missing Values...")
    missing_before = recovery_data.isnull().sum().sum()
    print(f"   Missing values before: {missing_before}")

    # For lag features, forward fill then backward fill
    recovery_data = recovery_data.fillna(method='ffill').fillna(method='bfill')

    # For remaining NaN, fill with median
    for col in recovery_data.select_dtypes(include=[np.number]).columns:
        if recovery_data[col].isnull().any():
            recovery_data[col].fillna(recovery_data[col].median(), inplace=True)

    missing_after = recovery_data.isnull().sum().sum()
    print(f"   Missing values after: {missing_after}")

    # Encode categorical variables
    if 'circuit_type' in recovery_data.columns:
        le = LabelEncoder()
        recovery_data['circuit_type_encoded'] = le.fit_transform(recovery_data['circuit_type'])

    # Separate features and target
    target = 'overall_recovery'
    exclude_cols = [target, 'circuit_type', 'timestamp'] if 'timestamp' in recovery_data.columns else [target, 'circuit_type']

    feature_cols = [col for col in recovery_data.columns if col not in exclude_cols]

    X = recovery_data[feature_cols]
    y = recovery_data[target]

    print(f"\n2. Feature Selection...")
    print(f"   Total features: {len(feature_cols)}")
    print(f"   Sample features: {feature_cols[:5]}")

    # Train-test split (80-20)
    print(f"\n3. Train-Test Split...")
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )
    print(f"   Training set: {X_train.shape}")
    print(f"   Test set: {X_test.shape}")

    # Feature scaling
    print(f"\n4. Feature Scaling...")
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    self.scalers['recovery'] = scaler

    # Train Random Forest
    print(f"\n5. Training Random Forest Regressor...")
    rf_model = RandomForestRegressor(
        n_estimators=200,
        max_depth=15,
        min_samples_split=10,
        min_samples_leaf=4,
        random_state=42,
        n_jobs=-1
    )

    rf_model.fit(X_train_scaled, y_train)

    # Train XGBoost
    print(f"\n6. Training XGBoost Regressor...")
    xgb_model = xgb.XGBRegressor(
        n_estimators=200,
        max_depth=8,
        learning_rate=0.05,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42
    )

    xgb_model.fit(X_train_scaled, y_train)

    # Evaluate both models
    print(f"\n7. Model Evaluation...")

    # Random Forest predictions
    y_pred_rf_train = rf_model.predict(X_train_scaled)
    y_pred_rf_test = rf_model.predict(X_test_scaled)

    # XGBoost predictions
    y_pred_xgb_train = xgb_model.predict(X_train_scaled)
    y_pred_xgb_test = xgb_model.predict(X_test_scaled)

    # Calculate metrics
    rf_results = {
        'train': {
            'MAE': mean_absolute_error(y_train, y_pred_rf_train),
            'RMSE': np.sqrt(mean_squared_error(y_train, y_pred_rf_train)),
            'R2': r2_score(y_train, y_pred_rf_train),
            'MAPE': np.mean(np.abs((y_train - y_pred_rf_train) / y_train)) * 100
        },
        'test': {
            'MAE': mean_absolute_error(y_test, y_pred_rf_test),
            'RMSE': np.sqrt(mean_squared_error(y_test, y_pred_rf_test)),
            'R2': r2_score(y_test, y_pred_rf_test),
            'MAPE': np.mean(np.abs((y_test - y_pred_rf_test) / y_test)) * 100
        }
    }

    xgb_results = {
        'train': {
            'MAE': mean_absolute_error(y_train, y_pred_xgb_train),
            'RMSE': np.sqrt(mean_squared_error(y_train, y_pred_xgb_train)),
            'R2': r2_score(y_train, y_pred_xgb_train),
            'MAPE': np.mean(np.abs((y_train - y_pred_xgb_train) / y_train)) * 100
        },
        'test': {
            'MAE': mean_absolute_error(y_test, y_pred_xgb_test),
            'RMSE': np.sqrt(mean_squared_error(y_test, y_pred_xgb_test)),
            'R2': r2_score(y_test, y_pred_xgb_test),
            'MAPE': np.mean(np.abs((y_test - y_pred_xgb_test) / y_test)) * 100
        }
    }

    # Display results
    print("\n" + "="*60)
    print("RANDOM FOREST RESULTS:")
    print("="*60)
    print("Training Set:")
    for metric, value in rf_results['train'].items():
        print(f"  {metric}: {value:.4f}")
    print("\nTest Set:")
    for metric, value in rf_results['test'].items():
        print(f"  {metric}: {value:.4f}")

    print("\n" + "="*60)
    print("XGBOOST RESULTS:")
    print("="*60)
    print("Training Set:")
    for metric, value in xgb_results['train'].items():
        print(f"  {metric}: {value:.4f}")
    print("\nTest Set:")
    for metric, value in xgb_results['test'].items():
        print(f"  {metric}: {value:.4f}")

    # Select best model
    if rf_results['test']['R2'] > xgb_results['test']['R2']:
        best_model = rf_model
        best_model_name = 'Random Forest'
        best_results = rf_results
        y_pred_test = y_pred_rf_test
    else:
        best_model = xgb_model
        best_model_name = 'XGBoost'
        best_results = xgb_results
        y_pred_test = y_pred_xgb_test

    print(f"\n✓ Best Model: {best_model_name} (Test R² = {best_results['test']['R2']:.4f})")

    # Feature importance
    if hasattr(best_model, 'feature_importances_'):
        feature_importance = pd.DataFrame({
            'feature': feature_cols,
            'importance': best_model.feature_importances_
        }).sort_values('importance', ascending=False)

        print(f"\nTop 10 Most Important Features:")
        print(feature_importance.head(10).to_string(index=False))

    # Store results
    self.models['recovery'] = {
        'model': best_model,
        'model_name': best_model_name,
        'features': feature_cols,
        'results': best_results,
        'feature_importance': feature_importance if hasattr(best_model, 'feature_importances_') else None
    }

    # Visualizations
    self.plot_recovery_results(y_test, y_pred_test, best_results, best_model_name)

    return best_model, best_results

def plot_recovery_results(self, y_test, y_pred, results, model_name):
    """Plot recovery model results"""
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))

    # Actual vs Predicted
    axes[0, 0].scatter(y_test, y_pred, alpha=0.5, s=20)
    axes[0, 0].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()],
                   'r--', linewidth=2, label='Perfect Prediction')
    axes[0, 0].set_xlabel('Actual Recovery')
    axes[0, 0].set_ylabel('Predicted Recovery')
    axes[0, 0].set_title(f'{model_name}: Actual vs Predicted\nR² = {results["test"]["R2"]:.4f}')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)

    # Residuals
    residuals = y_test - y_pred
    axes[0, 1].scatter(y_pred, residuals, alpha=0.5, s=20)
    axes[0, 1].axhline(y=0, color='r', linestyle='--', linewidth=2)
    axes[0, 1].set_xlabel('Predicted Recovery')
    axes[0, 1].set_ylabel('Residuals')
    axes[0, 1].set_title('Residual Plot')
    axes[0, 1].grid(True, alpha=0.3)

    # Residual distribution
    axes[1, 0].hist(residuals, bins=50, edgecolor='black', alpha=0.7)
    axes[1, 0].axvline(x=0, color='r', linestyle='--', linewidth=2)
    axes[1, 0].set_xlabel('Residual Value')
    axes[1, 0].set_ylabel('Frequency')
    axes[1, 0].set_title(f'Residual Distribution\nMAE = {results["test"]["MAE"]:.4f}')
    axes[1, 0].grid(True, alpha=0.3)

    # Error distribution
    errors = np.abs(residuals)
    axes[1, 1].hist(errors, bins=50, edgecolor='black', alpha=0.7, color='orange')
    axes[1, 1].set_xlabel('Absolute Error')
    axes[1, 1].set_ylabel('Frequency')
    axes[1, 1].set_title(f'Error Distribution\nRMSE = {results["test"]["RMSE"]:.4f}')
    axes[1, 1].grid(True, alpha=0.3)

    plt.tight_layout()
    plt.savefig('./model1_recovery_results.png', dpi=300, bbox_inches='tight')
    print(f"\n✓ Plots saved to: ./model1_recovery_results.png")
    plt.show()


In [ ]:
# MODEL 2: EQUIPMENT FAILURE PREDICTION - DATA PREPARATION

def prepare_equipment_failure_data(datasets):
        """Prepare data for equipment failure prediction"""
        print("MODEL 2: EQUIPMENT FAILURE PREDICTION - DATA PREPARATION")
        print("="*80)

        if 'equipment' not in datasets:
            raise ValueError("Equipment dataset not found!")

        equipment_df = datasets['equipment'].copy()

        # Create failure target variable (failure within 30 days)
        # Using failure_probability as proxy: >0.3 = high risk of failure
        equipment_df['failure_within_30_days'] = (equipment_df['failure_probability'] > 0.3).astype(int)

        print(f"✓ Created target variable: failure_within_30_days")
        print(f"  Failure cases: {equipment_df['failure_within_30_days'].sum()} ({equipment_df['failure_within_30_days'].mean()*100:.1f}%)")
        print(f"  Normal cases: {(equipment_df['failure_within_30_days']==0).sum()} ({(1-equipment_df['failure_within_30_days'].mean())*100:.1f}%)")

        # Select features for failure prediction
        failure_features = [
            # Core health metrics
            'health_score', 'operating_hours', 'rul_days',
            # Condition monitoring
            'vibration_rms', 'temperature_c', 'power_factor',
            # Engineered features
            'health_degradation_rate', 'equipment_age_factor',
            'vibration_severity_index', 'vibration_health_ratio',
            'temperature_deviation', 'thermal_stress_index',
            'power_factor_degradation', 'efficiency_loss_estimate',
            # Equipment specific
            'wear_rate_pct', 'health_wear_product', 'maintenance_urgency_score'
        ]

        # Lag features if available
        lag_features = [col for col in equipment_df.columns if 'lag' in col or 'rolling' in col]
        failure_features.extend(lag_features[:10])  # Add first 10 lag features

        # Check which features exist
        available_features = [f for f in failure_features if f in equipment_df.columns]

        print(f"\n✓ Selected {len(available_features)} features for failure prediction")
        print(f"  Sample features: {available_features[:5]}")

        # Create final dataset
        failure_data = equipment_df[available_features + ['failure_within_30_days', 'equipment_type']].copy()

        print(f"\n✓ Dataset shape: {failure_data.shape}")

        return failure_data

    def train_failure_prediction_model(self, failure_data):
        """Train equipment failure prediction model"""
        print("\n" + "="*80)
        print("MODEL 2: TRAINING FAILURE PREDICTION MODEL")
        print("="*80)

        # Handle missing values
        print("\n1. Handling Missing Values...")
        missing_before = failure_data.isnull().sum().sum()
        print(f"   Missing values before: {missing_before}")

        failure_data = failure_data.fillna(method='ffill').fillna(method='bfill')

        for col in failure_data.select_dtypes(include=[np.number]).columns:
            if failure_data[col].isnull().any():
                failure_data[col].fillna(failure_data[col].median(), inplace=True)

        missing_after = failure_data.isnull().sum().sum()
        print(f"   Missing values after: {missing_after}")

        # Encode equipment type
        if 'equipment_type' in failure_data.columns:
            le = LabelEncoder()
            failure_data['equipment_type_encoded'] = le.fit_transform(failure_data['equipment_type'])

        # Separate features and target
        target = 'failure_within_30_days'
        exclude_cols = [target, 'equipment_type', 'timestamp'] if 'timestamp' in failure_data.columns else [target, 'equipment_type']

        feature_cols = [col for col in failure_data.columns if col not in exclude_cols]

        X = failure_data[feature_cols]
        y = failure_data[target]

        print(f"\n2. Feature Selection...")
        print(f"   Total features: {len(feature_cols)}")

        # Train-test split (stratified)
        print(f"\n3. Train-Test Split (Stratified)...")
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.2, random_state=42, stratify=y
        )
        print(f"   Training set: {X_train.shape}")
        print(f"   Test set: {X_test.shape}")
        print(f"   Training failures: {y_train.sum()} ({y_train.mean()*100:.1f}%)")
        print(f"   Test failures: {y_test.sum()} ({y_test.mean()*100:.1f}%)")

        # Feature scaling
        print(f"\n4. Feature Scaling...")
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)
        self.scalers['failure'] = scaler

        # Train Random Forest Classifier
        print(f"\n5. Training Random Forest Classifier...")
        rf_clf = RandomForestClassifier(
            n_estimators=200,
            max_depth=15,
            min_samples_split=10,
            min_samples_leaf=4,
            class_weight='balanced',  # Handle imbalanced data
            random_state=42,
            n_jobs=-1
        )

        rf_clf.fit(X_train_scaled, y_train)

        # Train XGBoost Classifier
        print(f"\n6. Training XGBoost Classifier...")

        # Calculate scale_pos_weight for imbalanced data
        scale_pos_weight = (y_train == 0).sum() / (y_train == 1).sum()

        xgb_clf = xgb.XGBClassifier(
            n_estimators=200,
            max_depth=8,
            learning_rate=0.05,
            subsample=0.8,
            colsample_bytree=0.8,
            scale_pos_weight=scale_pos_weight,
            random_state=42
        )

        xgb_clf.fit(X_train_scaled, y_train)

        # Evaluate both models
        print(f"\n7. Model Evaluation...")

        # Random Forest predictions
        y_pred_rf_train = rf_clf.predict(X_train_scaled)
        y_pred_rf_test = rf_clf.predict(X_test_scaled)
        y_pred_rf_proba = rf_clf.predict_proba(X_test_scaled)[:, 1]

        # XGBoost predictions
        y_pred_xgb_train = xgb_clf.predict(X_train_scaled)
        y_pred_xgb_test = xgb_clf.predict(X_test_scaled)
        y_pred_xgb_proba = xgb_clf.predict_proba(X_test_scaled)[:, 1]

        # Calculate metrics
        rf_results = {
            'train': {
                'Accuracy': accuracy_score(y_train, y_pred_rf_train),
                'Precision': precision_score(y_train, y_pred_rf_train, zero_division=0),
                'Recall': recall_score(y_train, y_pred_rf_train, zero_division=0),
                'F1': f1_score(y_train, y_pred_rf_train, zero_division=0)
            },
            'test': {
                'Accuracy': accuracy_score(y_test, y_pred_rf_test),
                'Precision': precision_score(y_test, y_pred_rf_test, zero_division=0),
                'Recall': recall_score(y_test, y_pred_rf_test, zero_division=0),
                'F1': f1_score(y_test, y_pred_rf_test, zero_division=0),
                'ROC-AUC': roc_auc_score(y_test, y_pred_rf_proba)
            },
            'confusion_matrix': confusion_matrix(y_test, y_pred_rf_test),
            'y_pred_proba': y_pred_rf_proba
        }

        xgb_results = {
            'train': {
                'Accuracy': accuracy_score(y_train, y_pred_xgb_train),
                'Precision': precision_score(y_train, y_pred_xgb_train, zero_division=0),
                'Recall': recall_score(y_train, y_pred_xgb_train, zero_division=0),
                'F1': f1_score(y_train, y_pred_xgb_train, zero_division=0)
            },
            'test': {
                'Accuracy': accuracy_score(y_test, y_pred_xgb_test),
                'Precision': precision_score(y_test, y_pred_xgb_test, zero_division=0),
                'Recall': recall_score(y_test, y_pred_xgb_test, zero_division=0),
                'F1': f1_score(y_test, y_pred_xgb_test, zero_division=0),
                'ROC-AUC': roc_auc_score(y_test, y_pred_xgb_proba)
            },
            'confusion_matrix': confusion_matrix(y_test, y_pred_xgb_test),
            'y_pred_proba': y_pred_xgb_proba
        }

        # Display results
        print("\n" + "="*60)
        print("RANDOM FOREST CLASSIFIER RESULTS:")
        print("="*60)
        print("Training Set:")
        for metric, value in rf_results['train'].items():
            print(f"  {metric}: {value:.4f}")
        print("\nTest Set:")
        for metric, value in rf_results['test'].items():
            print(f"  {metric}: {value:.4f}")
        print("\nConfusion Matrix (Test):")
        print(rf_results['confusion_matrix'])

        print("\n" + "="*60)
        print("XGBOOST CLASSIFIER RESULTS:")
        print("="*60)
        print("Training Set:")
        for metric, value in xgb_results['train'].items():
            print(f"  {metric}: {value:.4f}")
        print("\nTest Set:")
        for metric, value in xgb_results['test'].items():
            print(f"  {metric}: {value:.4f}")
        print("\nConfusion Matrix (Test):")
        print(xgb_results['confusion_matrix'])

        # Select best model (prioritize recall for safety)
        if rf_results['test']['Recall'] > xgb_results['test']['Recall']:
            best_model = rf_clf
            best_model_name = 'Random Forest'
            best_results = rf_results
            y_pred_test = y_pred_rf_test
            y_pred_proba = y_pred_rf_proba
        else:
            best_model = xgb_clf
            best_model_name = 'XGBoost'
            best_results = xgb_results
            y_pred_test = y_pred_xgb_test
            y_pred_proba = y_pred_xgb_proba

        print(f"\n✓ Best Model: {best_model_name} (Recall = {best_results['test']['Recall']:.4f})")
        print(f"  Note: Recall prioritized to minimize missed failures (safety critical)")

        # Feature importance
        if hasattr(best_model, 'feature_importances_'):
            feature_importance = pd.DataFrame({
                'feature': feature_cols,
                'importance': best_model.feature_importances_
            }).sort_values('importance', ascending=False)

            print(f"\nTop 10 Most Important Features for Failure Prediction:")
            print(feature_importance.head(10).to_string(index=False))

        # Store results
        self.models['failure'] = {
            'model': best_model,
            'model_name': best_model_name,
            'features': feature_cols,
            'results': best_results,
            'feature_importance': feature_importance if hasattr(best_model, 'feature_importances_') else None
        }

        # Visualizations
        self.plot_failure_results(y_test, y_pred_test, y_pred_proba, best_results, best_model_name)

        return best_model, best_results

    def plot_failure_results(self, y_test, y_pred, y_pred_proba, results, model_name):
        """Plot failure prediction model results"""
        fig, axes = plt.subplots(2, 2, figsize=(16, 12))

        # Confusion Matrix
        cm = results['confusion_matrix']
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[0, 0],
                   xticklabels=['No Failure', 'Failure'],
                   yticklabels=['No Failure', 'Failure'])
        axes[0, 0].set_xlabel('Predicted')
        axes[0, 0].set_ylabel('Actual')
        axes[0, 0].set_title(f'{model_name}: Confusion Matrix\nAccuracy = {results["test"]["Accuracy"]:.4f}')

        # ROC Curve
        fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)
        axes[0, 1].plot(fpr, tpr, linewidth=2, label=f'ROC (AUC = {results["test"]["ROC-AUC"]:.4f})')
        axes[0, 1].plot([0, 1], [0, 1], 'k--', linewidth=2, label='Random Classifier')
        axes[0, 1].set_xlabel('False Positive Rate')
        axes[0, 1].set_ylabel('True Positive Rate (Recall)')
        axes[0, 1].set_title('ROC Curve')
        axes[0, 1].legend()
        axes[0, 1].grid(True, alpha=0.3)

        # Precision-Recall Trade-off
        precision, recall, pr_thresholds = precision_recall_curve(y_test, y_pred_proba)
        axes[1, 0].plot(recall, precision, linewidth=2, color='green')
        axes[1, 0].set_xlabel('Recall')
        axes[1, 0].set_ylabel('Precision')
        axes[1, 0].set_title(f'Precision-Recall Curve\nF1 = {results["test"]["F1"]:.4f}')
        axes[1, 0].grid(True, alpha=0.3)

        # Prediction Probability Distribution
        axes[1, 1].hist(y_pred_proba[y_test == 0], bins=30, alpha=0.6, label='No Failure', color='blue')
        axes[1, 1].hist(y_pred_proba[y_test == 1], bins=30, alpha=0.6, label='Failure', color='red')
        axes[1, 1].axvline(x=0.5, color='black', linestyle='--', linewidth=2, label='Threshold (0.5)')
        axes[1, 1].set_xlabel('Predicted Probability')
        axes[1, 1].set_ylabel('Frequency')
        axes[1, 1].set_title('Prediction Probability Distribution')
        axes[1, 1].legend()
        axes[1, 1].grid(True, alpha=0.3)

        plt.tight_layout()
        plt.savefig('./model2_failure_results.png', dpi=300, bbox_inches='tight')
        print(f"\n✓ Plots saved to: ./model2_failure_results.png")
        plt.show()